In [1]:
# Setup plotting
import matplotlib.pyplot as plt
from learntools.deep_learning_intro.dltools import animate_sgd
plt.style.use('seaborn-whitegrid')
# Set Matplotlib defaults
plt.rc('figure', autolayout=True)
plt.rc('axes', labelweight='bold', labelsize='large',
       titleweight='bold', titlesize=18, titlepad=10)
plt.rc('animation', html='html5')

# Setup feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.deep_learning_intro.ex3 import *

2025-10-17 03:37:35.060089: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760672255.345370      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760672255.429015      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/tmp/ipykernel_13/3305082411.py:4: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-whitegrid')


In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.model_selection import train_test_split

fuel = pd.read_csv('../input/dl-course-data/fuel.csv')

X = fuel.copy()
# Remove target
y = X.pop('FE')

preprocessor = make_column_transformer(
    (StandardScaler(),
     make_column_selector(dtype_include=np.number)),
    (OneHotEncoder(sparse=False),
     make_column_selector(dtype_include=object)),
)

X = preprocessor.fit_transform(X)
y = np.log(y) # log transform target instead of standardizing

input_shape = [X.shape[1]]
print("Input shape: {}".format(input_shape))

Input shape: [50]


/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [3]:
fuel.head()

pd.DataFrame(X[:10,:]).head()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,0.913643,1.068005,0.524148,0.685653,-0.226455,0.391659,0.43492,0.463841,-0.447941,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.913643,1.068005,0.524148,0.685653,-0.226455,0.391659,0.43492,0.463841,-0.447941,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.530594,1.068005,0.524148,0.685653,-0.226455,0.391659,0.43492,0.463841,-0.447941,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.530594,1.068005,0.524148,0.685653,-0.226455,0.391659,0.43492,0.463841,-0.447941,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.296693,2.120794,0.524148,-1.458464,-0.226455,0.391659,0.43492,0.463841,-0.447941,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
import torch
import torch.nn as nn

class SimpleMLP(nn.Module):
    def __init__(self, input_size):
        super(SimpleMLP, self).__init__()
        self.layer1 = nn.Linear(input_size, 128)
        self.relu = nn.ReLU()
        self.layer2 = nn.Linear(128, 128)
        self.layer3 = nn.Linear(128, 64)
        self.output_layer = nn.Linear(64, 1)

    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.relu(self.layer2(x))
        x = self.relu(self.layer3(x))
        x = self.output_layer(x)
        return x

# Assuming 'input_size' is defined somewhere in your code, e.g., input_size = 784 for MNIST
input_size = 784  # Replace with your actual input dimension
model = SimpleMLP(input_size)

# 1) Add Loss and Optimizer

In [5]:
import torch.optim as optim

In [6]:
criterion = nn.L1Loss() # 'mae' (Mean Absolute Error) is L1 Loss in PyTorch

optimizer = optim.Adam(model.parameters(), lr=0.001) # Default Keras learning rate is usually 0.001

# 2) Train Model

In [7]:
import torch
from torch.utils.data import TensorDataset, DataLoader

input_size = input_shape[0]  # This will be 50
model = SimpleMLP(input_size)

# Convert numpy arrays to PyTorch Tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32).view(-1, 1)

# Create a TensorDataset and DataLoader for batching
dataset = TensorDataset(X_tensor, y_tensor)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True) # Adjust batch_size as needed

# Define training parameters
num_epochs = 50 # Or a different number

# Training loop
for epoch in range(num_epochs):
    for X_batch, y_batch in dataloader:
        # 1. Zero the gradients
        optimizer.zero_grad()

        # 2. Forward pass
        outputs = model(X_batch)

        # 3. Calculate the loss
        loss = criterion(outputs, y_batch)

        # 4. Backward pass
        loss.backward()

        # 5. Update weights
        optimizer.step()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

# After the loop, you can evaluate the model on new data.

Epoch [1/50], Loss: 3.5400
Epoch [2/50], Loss: 3.5045
Epoch [3/50], Loss: 3.5719
Epoch [4/50], Loss: 3.5056
Epoch [5/50], Loss: 3.4754
Epoch [6/50], Loss: 3.4861
Epoch [7/50], Loss: 3.4857
Epoch [8/50], Loss: 3.5191
Epoch [9/50], Loss: 3.4447
Epoch [10/50], Loss: 3.5106
Epoch [11/50], Loss: 3.5719
Epoch [12/50], Loss: 3.5096
Epoch [13/50], Loss: 3.4901
Epoch [14/50], Loss: 3.5348
Epoch [15/50], Loss: 3.4786
Epoch [16/50], Loss: 3.4534
Epoch [17/50], Loss: 3.4891
Epoch [18/50], Loss: 3.4459
Epoch [19/50], Loss: 3.4529
Epoch [20/50], Loss: 3.4685
Epoch [21/50], Loss: 3.5186
Epoch [22/50], Loss: 3.5717
Epoch [23/50], Loss: 3.5617
Epoch [24/50], Loss: 3.4385
Epoch [25/50], Loss: 3.4931
Epoch [26/50], Loss: 3.5994
Epoch [27/50], Loss: 3.5188
Epoch [28/50], Loss: 3.4105
Epoch [29/50], Loss: 3.5799
Epoch [30/50], Loss: 3.5574
Epoch [31/50], Loss: 3.4591
Epoch [32/50], Loss: 3.4969
Epoch [33/50], Loss: 3.5497
Epoch [34/50], Loss: 3.5332
Epoch [35/50], Loss: 3.4801
Epoch [36/50], Loss: 3.3962
E